### Try-it 9.2: Predicting Wages

This activity is meant to summarize your work with regularized regression models.  You will use your earlier work with data preparation and pipelines together with what you've learned with grid searches to determine an optimal model.  In addition to the prior strategies, this example is an excellent opportunity to utilize the `TransformedTargetRegressor` estimator in scikitlearn.

### The Data

This dataset is loaded from the openml resource library.  Originally from census data, the data contains wage and demographic information on 534 individuals. From the dataset documentation [here](https://www.openml.org/d/534)

```
The Current Population Survey (CPS) is used to supplement census information between census years. These data consist of a random sample of 534 persons from the CPS, with information on wages and other characteristics of the workers, including sex, number of years of education, years of work experience, occupational status, region of residence and union membership. 
```

In [1]:
from sklearn.datasets import fetch_openml

In [2]:
wages = fetch_openml(data_id=534, as_frame=True)

In [3]:
wages.frame.head()

,EDUCATION,SOUTH,SEX,EXPERIENCE,UNION,WAGE,AGE,RACE,OCCUPATION,SECTOR,MARR
0,8,no,female,21,not_member,5.10,35,Hispanic,Other,Manufacturing,Married
1,9,no,female,42,not_member,4.95,57,White,Other,Manufacturing,Married
2,12,no,male,1,not_member,6.67,19,White,Other,Manufacturing,Unmarried
3,12,no,male,4,not_member,4.00,22,White,Other,Other,Unmarried
4,12,no,male,17,not_member,7.50,35,White,Other,Other,Married


#### Task

Build regression models to predict `WAGE`.  Incorporate the categorical features and transform the target using a logarithm.  Build `Ridge` models and consider some different amounts of regularization.  

After fitting your model, interpret the model and try to understand what features led to higher wages.  Consider using `permutation_importance` that you encountered in module 8.  Discuss your findings in the class forum.

For an in depth example discussing the perils of interpreting the coefficients see the example in scikitlearn examples [here](https://scikit-learn.org/stable/auto_examples/inspection/plot_linear_model_coefficient_interpretation.html).

In [4]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SequentialFeatureSelector
import numpy as np
import plotly.express as px
import pandas as pd
import warnings

In [6]:
df = wages.frame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 534 entries, 0 to 533
Data columns (total 11 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   EDUCATION   534 non-null    int64   
 1   SOUTH       534 non-null    category
 2   SEX         534 non-null    category
 3   EXPERIENCE  534 non-null    int64   
 4   UNION       534 non-null    category
 5   WAGE        534 non-null    float64 
 6   AGE         534 non-null    int64   
 7   RACE        534 non-null    category
 8   OCCUPATION  534 non-null    category
 9   SECTOR      534 non-null    category
 10  MARR        534 non-null    category
dtypes: category(7), float64(1), int64(3)
memory usage: 21.4 KB


### Data Proprecessing

In [8]:
df_num = df.select_dtypes(include = 'number')
df_num

,EDUCATION,EXPERIENCE,WAGE,AGE
0,8,21,5.10,35
1,9,42,4.95,57
2,12,1,6.67,19
3,12,4,4.00,22
4,12,17,7.50,35
...,...,...,...,...
529,18,5,11.36,29
530,12,33,6.10,51
531,17,25,23.25,48
532,12,13,19.88,31


In [11]:
df_obj = df.select_dtypes(exclude = 'number')
df_obj

,SOUTH,SEX,UNION,RACE,OCCUPATION,SECTOR,MARR
0,no,female,not_member,Hispanic,Other,Manufacturing,Married
1,no,female,not_member,White,Other,Manufacturing,Married
2,no,male,not_member,White,Other,Manufacturing,Unmarried
3,no,male,not_member,White,Other,Other,Unmarried
4,no,male,not_member,White,Other,Other,Married
...,...,...,...,...,...,...,...
529,no,male,not_member,White,Professional,Other,Unmarried
530,no,female,not_member,Other,Professional,Other,Married
531,no,female,member,Other,Professional,Other,Married
532,yes,male,member,White,Professional,Other,Married


In [13]:
df_obj['SOUTH'].unique()

['no', 'yes']
Categories (2, object): ['no', 'yes']

In [19]:
for i in df_obj:
    print(i,df_obj[i].nunique())

SOUTH 2
SEX 2
UNION 2
RACE 3
OCCUPATION 6
SECTOR 3
MARR 2


In [16]:
df_binary = df_obj[['SOUTH','SEX','UNION','MARR']]
df_binary.head()

,SOUTH,SEX,UNION,MARR
0,no,female,not_member,Married
1,no,female,not_member,Married
2,no,male,not_member,Unmarried
3,no,male,not_member,Unmarried
4,no,male,not_member,Married


In [17]:
df_multi = df_obj[['RACE','OCCUPATION','SECTOR']]
df_multi.head()

,RACE,OCCUPATION,SECTOR
0,Hispanic,Other,Manufacturing
1,White,Other,Manufacturing
2,White,Other,Manufacturing
3,White,Other,Other
4,White,Other,Other
